In [1]:
from Model_Training import *
from Cleaner import Cleaner
import plotly.express as px
import warnings
import pandas as pd
from EmpiricDistribution import EmpiricDistribution
from datetime import datetime, time
import pickle # Para guardar los modelos entrenados
from datetime import time 
import matplotlib.pyplot as plt
import numpy as np
import random
from sympy import Symbol
from datetime import time
import random
from dataclasses import dataclass

warnings.filterwarnings('ignore')

buses_model = Buses_model()
pax_model = Pax_model()

#INTIAL SETUP
N_BUSES = 2

pax_model.generate_model(n_buses=N_BUSES)

### Buses

In [2]:
#Uncomment this line if you dont have the bus_results_times.csv
#buses_model.generate_simulation(n_samples=BUS_SAMPLES)

### DISCRETE SIMULATION

In [ ]:
@dataclass
class Stop:
    name: str
    buses_arrivals: dict
    pax_arrivals: list

@dataclass
class Bus:
    id: str
    start_second: float
    start_place: str
    cost: float
    capacity: int

@dataclass
class Campus:
    stops: dict

start_time = '07:00'
end_time = '09:30'

eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13)

glaxo = Stop(
    name='GLAXO', 
    buses_arrivals=buses_model.generate_simulation(target_trayecto='A2-GLAXO', target_interval=(start_time, end_time), n_samples=1000),
    pax_arrivals=pax_model.generate_simulation(target_trayecto='A2-GLAXO', target_interval=(start_time, end_time), n_samples=5000)
    )